# Updating the SVD

---

In many applications which are based on the SVD, arrival of new data requires SVD of the new matrix. Instead of computing from scratch, existing SVD can be updated.

## Prerequisites

The reader should be familiar with concepts of singular values and singular vectors, related perturbation theory, and algorithms.
 
## Competences 

The reader should be able to recognise applications where SVD updating can be sucessfully applied and apply it.

---

## Facts

For more details see
[M. Gu and S. C. Eisenstat, A Stable and Fast Algorithm for Updating the Singular Value Decomposition][GE93]
and [M. Brand, Fast low-rank modifications of the thin singular value decomposition][Bra06]
and the references therein.

[GE93]: http://www.cs.yale.edu/publications/techreports/tr966.pdf "M. Gu and S. C. Eisenstat, 'A Stable and Fast Algorithm for Updating the Singular Value Decomposition', Tech.report, Yale University, 1993."

[Bra06]: http://www.sciencedirect.com/science/article/pii/S0024379505003812 "M. Brand, 'Fast low-rank modifications of the thin singular value decomposition', Linear Algebra and its Appl, 415 (20-30) 2006."

1. Let $A\in\mathbb{R}^{m\times n}$ with $m\geq n$ and $\mathop{\mathrm{rank}}(A)=n$, and  let $A=U\Sigma V^T$ be its SVD.
   Let $a\in\mathbb{R}^{n}$ be a vector, and let $\tilde A=\begin{bmatrix} A \\ a^T\end{bmatrix}$. Then
   $$\begin{bmatrix} A \\ a^T\end{bmatrix} =\begin{bmatrix} U & \\ & 1 \end{bmatrix} 
   \begin{bmatrix} \Sigma \\ a^TV \end{bmatrix}  V^T.
   $$
   Let $\begin{bmatrix} \Sigma \\ a^T V \end{bmatrix} = \bar U \bar \Sigma \bar V^T$ be the SVD of the half-arrowhead matrix. _This SVD can be computed in $O(n^2)$ operations._ Then 
   $$\begin{bmatrix} A \\ a^T\end{bmatrix} =
   \begin{bmatrix} U & \\ & 1 \end{bmatrix} \bar U \bar\Sigma \bar V^T V^T \equiv
   \tilde U \bar \Sigma \tilde V^T
   $$
   is the SVD of $\tilde A$. 
   
2. Direct computation of $\tilde U$ and $\tilde V$ requires $O(mn^2)$ and $O(n^3)$ operations. However, these multiplications can be performed using Fast Multipole Method. This is not (yet) implemented in Julia and is "not for the timid" (quote by Steven G. Johnson).

3. If $m<n$ and $\mathop{\mathrm{rank}}(A)=n$, then
   $$
   \begin{bmatrix} A \\ a^T\end{bmatrix} =\begin{bmatrix} U & \\ & 1 \end{bmatrix} 
   \begin{bmatrix} \Sigma & 0 \\ a^T V & \beta\end{bmatrix} \begin{bmatrix} V^T \\ v^T \end{bmatrix},
   $$
   where $\beta=\sqrt{\|a\|_2^2-\|V^T a\|_2^2}$ and $v=(I-VV^T)a$. Notice that $V^Tv=0$ by construction.
   Let $\begin{bmatrix} \Sigma & 0 \\ a^T V &  \beta\end{bmatrix} = \bar U \bar \Sigma \bar V^T$ be the SVD of 
   the half-arrowhead matrix. Then 
   $$\begin{bmatrix} A \\ a^T\end{bmatrix} =
   \begin{bmatrix} U & \\ & 1 \end{bmatrix} \bar U \bar\Sigma \bar V^T \begin{bmatrix} V^T \\ v^T \end{bmatrix}
   \equiv \tilde U \bar \Sigma \tilde V^T
   $$
   is the SVD of $\tilde A$.
   
3. Adding a column $a$ to $A$ is equivalent to adding a row $a^T$ to $A^T$.

3. If $\mathop{\mathrm{rank}}(A)<\min\{m,n\}$ or if we are using SVD approximation of rank $r$, and if we want to keep the rank of the approximation (this is the common case in practice), then the formulas in Fact 1 hold approximately. More precisely, the updated rank $r$ approximation is __not__ what we would get by computing the approximation of rank $r$ of the updated matrix, but is sufficient in many applications. 

### Example - Adding row to a tall matrix

If $m>=n$, adding row does not increase the size of $\Sigma$.

In [1]:
using Arrowhead

In [2]:
function mySVDaddrow{T}(svdA::Tuple,a::Vector{T})
    # Create the transposed half-arrowhead
    m,r,n=size(svdA[1],1),length(svdA[2]),size(svdA[3],1)
    b=svdA[3]'*a
    if m>=n || r<m
        M=HalfArrow(svdA[2],b)
    else
        β=sqrt(vecnorm(a)^2-vecnorm(b)^2)
        M=HalfArrow(svdA[2],[b;β])
    end
    tols=[1e2,1e2,1e2,1e2]
    U,σ,V=svd(M,tols)
    # Return the updated SVD
    if m>=n || r<m
        return [svdA[1] zeros(T,m); zeros(T,1,r) one(T)]*V, σ, svdA[3]*U
    else
        # Need one more row of svdA[3] - v is orthogonal projection
        v=a-svdA[3]*b
        v=v/norm(v)
        return [svdA[1] zeros(T,m); zeros(T,1,r) one(T)]*V, σ, [svdA[3] v]*U
    end
end

mySVDaddrow (generic function with 1 method)

In [3]:
A=rand(10,6)
a=rand(6)

6-element Array{Float64,1}:
 0.00974734
 0.317398  
 0.0364821 
 0.372837  
 0.269291  
 0.798855  

In [4]:
svdA=svd(A)

(
[-0.297815 -0.375132 … -0.417715 0.569589; -0.330464 0.326842 … -0.135102 -0.112997; … ; -0.369931 -0.417202 … 0.340163 0.122; -0.186689 0.178731 … 0.665139 0.493761],

[4.08794,1.21578,1.14914,0.867703,0.698743,0.476594],
[-0.437489 0.0968342 … -0.0993064 -0.601206; -0.318728 -0.673347 … 0.661561 0.0681239; … ; -0.344374 0.666724 … 0.499734 0.13083; -0.41221 0.105243 … -0.177075 0.698884])

In [5]:
typeof(svdA)

Tuple{Array{Float64,2},Array{Float64,1},Array{Float64,2}}

In [6]:
U,σ,V=mySVDaddrow(svdA,a)

Remedy 3 


(
[-0.29467 -0.379224 … -0.279361 -0.445403; -0.323916 0.340261 … 0.0960956 -0.127276; … ; -0.184414 0.186134 … -0.435461 0.631594; -0.184376 -0.048331 … -0.729542 -0.0501926],

[4.15763,1.21671,1.15667,0.874184,0.757578,0.698422],
[-0.423198 0.0913728 … 0.608335 -0.0480599; -0.322243 -0.675542 … -0.0995015 0.655065; … ; -0.344539 0.674899 … -0.26025 0.482996; -0.434232 0.0663593 … -0.559475 -0.228605])

In [7]:
# Check the residual and orthogonality
norm([A;a']*V-U*diagm(σ)), norm(U'*U-I), norm(V'*V-I)

(1.8894489268057245e-15,9.695165926855932e-16,1.3931207765288609e-15)

### Example - Adding row to a flat matrix

In [8]:
# Now flat matrix
A=rand(6,10)
a=rand(10)
svdA=svd(A)

(
[-0.472629 -0.235703 … -0.613148 0.374261; -0.306033 -0.693233 … 0.390215 -0.240224; … ; -0.392951 0.43789 … 0.273343 0.538759; -0.382543 0.509278 … 0.123266 -0.568327],

[4.09851,1.2037,1.06981,0.853926,0.764236,0.367927],
[-0.366563 0.35742 … -0.109813 0.204003; -0.326924 0.416155 … -0.0737274 -0.332693; … ; -0.281541 -0.289135 … 0.154429 -0.149111; -0.23192 -0.189867 … 0.475782 0.246146])

In [9]:
U,σ,V=mySVDaddrow(svdA,a)
norm([A;a']*V-U*diagm(σ)), norm(U'*U-I), norm(V'*V-I)

(2.046352241566695e-14,1.5037343132422024e-15,1.2585675962318527e-15)

### Example - Adding columns

This can be viewed as adding rows to the transposed matrix, an elegant one-liner in Julia.

In [10]:
function mySVDaddcol{T}(svdA::Tuple,a::Vector{T})
    reverse(mySVDaddrow(reverse(svdA),a))
end 

mySVDaddcol (generic function with 1 method)

In [11]:
# Tall matrix
A=rand(10,6)
a=rand(10)
svdA=svd(A)
U,σ,V=mySVDaddcol(svdA,a)
norm([A a]*V-U*diagm(σ)), norm(U'*U-I), norm(V'*V-I)

(1.6025018700859276e-15,6.715088528880365e-15,1.9253181243819344e-15)

In [12]:
# Flat matrix
A=rand(6,10)
a=rand(6)
svdA=svd(A)
U,σ,V=mySVDaddcol(svdA,a)
norm([A a]*V-U*diagm(σ)), norm(U'*U-I), norm(V'*V-I)

(1.8549040443514024e-15,8.826013242280194e-16,1.2075260880394839e-15)

In [13]:
# Square matrix
A=rand(10,10)
a=rand(10);
svdA=svd(A);

In [14]:
U,σ,V=mySVDaddrow(svdA,a)
norm([A;a']*V-U*diagm(σ)), norm(U'*U-I), norm(V'*V-I)

(9.10725749031954e-15,1.790404539287269e-15,1.7572149809141083e-15)

In [15]:
U,σ,V=mySVDaddcol(svdA,a)
norm([A a]*V-U*diagm(σ)), norm(U'*U-I), norm(V'*V-I)

Remedy 3 


(9.291893491473034e-15,1.439029309831976e-15,1.632411162262733e-15)

### Example - Updating a low rank approximation


In [16]:
# Adding row to a tall matrix
A=rand(10,6)
svdA=svd(A)
a=rand(6)
# Rank of the approximation
r=4

4

In [17]:
svdAr=(svdA[1][:,1:r], svdA[2][1:r],svdA[3][:,1:r])
U,σ,V=mySVDaddrow(svdAr,a)
norm([A;a']-U*diagm(σ)*V'), svdvals([A;a']), σ

(0.6600073111821658,[4.71192,1.4163,1.11822,0.700675,0.651378,0.271346],[4.71096,1.41576,1.11774,0.700578])

In [18]:
# Adding row to a flat matrix
A=rand(6,10)
svdA=svd(A)
a=rand(10)
# Rank of the approximation
r=4

4

In [19]:
svdAr=(svdA[1][:,1:r], svdA[2][1:r],svdA[3][:,1:r])
U,σ,V=mySVDaddrow(svdAr,a)
norm([A;a']-U*diagm(σ)*V'), svdvals([A;a']), σ

(0.8411846014870148,[4.1918,1.3001,1.00868,0.675576,0.595288,0.499218,0.228495],[4.17065,1.29993,0.96634,0.600488])